# Benchmark 

In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
import os
import numpy as np
import scanpy as sc
import time
from pathlib import Path
import torch
import concord as ccd
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import matplotlib as mpl
from scipy import sparse

# from matplotlib import font_manager, rcParams
# custom_rc = {
#     'font.family': 'Arial',  # Set the desired font for this plot
# }

# mpl.rcParams['svg.fonttype'] = 'none'
# mpl.rcParams['pdf.fonttype'] = 42

In [ ]:
proj_name = "dkd_Wilson"
file_name = "dkd_Wilson"
file_suffix = time.strftime('%b%d-%H%M')

# save_dir = Path(f"../save/{proj_name}-{file_suffix}")
# save_dir.mkdir(parents=True, exist_ok=True)

data_dir = Path(f"../data/{proj_name}")
data_dir.mkdir(parents=True, exist_ok=True)

seed = 0
ccd.ul.set_seed(seed)

In [ ]:
# load the data
adata = sc.read_h5ad(data_dir / f"{file_name}.h5ad")

In [ ]:
adata

In [ ]:
sc.pp.highly_variable_genes(adata, flavor='seurat_v3', n_top_genes=5000, subset=False)
sc.tl.pca(adata, n_comps=300, svd_solver='arpack', use_highly_variable=True)
adata = adata[:, adata.var.highly_variable].copy()

In [ ]:
# Add raw counts to adata.layers["counts"] after HVG filtering
if adata.raw is not None:
    adata.layers["counts"] = adata.raw[:, adata.var_names].X.astype(np.int32)
    print("✅ counts layer set from adata.raw (shape matched to HVGs)")
else:
    raise ValueError("❌ adata.raw is None — cannot set counts layer.")


In [ ]:
import numpy as np

zero_expression = np.array(adata.X.sum(axis=1)).flatten() == 0
n_zero_cells = np.sum(zero_expression)

print(f"❗ {n_zero_cells} cells have 0 expression in `adata.X`")


In [ ]:
# import pandas as pd

batch_key = 'donor_id'  # or your actual batch column in adata.obs
for batch in adata.obs[batch_key].unique():
    sub = adata[adata.obs[batch_key] == batch]
    X = sub.X.toarray() if sparse.issparse(sub.X) else sub.X
    zero_var = np.sum(np.var(X, axis=0) == 0)
    print(f"{batch}: {zero_var} zero-variance genes")


In [ ]:
min_val = adata.X.min()
print(f"🔍 Minimum value in adata.X: {min_val}")

max_val = adata.X.max()
print(f"🔍 Maximum value in adata.X: {max_val}")

if min_val < 0:
    print("⚠️ adata.X is scaled — contains negative values.")
else:
    print("✅ adata.X is non-negative — likely log1p transformed, not scaled.")


In [ ]:
counts = adata.layers["counts"]
if counts is None:
    print("❌ adata.layers['counts'] does not exist.")
else:
    # Check min and integer-ness
    if sparse.issparse(counts):
        counts = counts.data  # only nonzero values
    else:
        counts = counts.flatten()

    min_val = counts.min()
    is_integer = np.allclose(counts, np.round(counts))

    print(f"🔍 counts layer — min: {min_val}, integer-like: {is_integer}")
    if min_val < 0:
        print("❌ counts layer contains negative values!")
    elif not is_integer:
        print("❌ counts layer contains non-integer values!")
    else:
        print("✅ counts layer looks valid: non-negative integers.")


In [ ]:
# save the adata object
adata.write_h5ad(data_dir / f"{file_name}_preprocessed.h5ad")
print(f"✅ Preprocessed data saved to {data_dir / f'{file_name}_preprocessed.h5ad'}")

In [ ]:
adata = sc.read_h5ad(data_dir / f"{file_name}_preprocessed.h5ad")

In [ ]:
import subprocess, json
methods = ["scvi", "harmony", "scanorama", "liger", "unintegrated", "concord_hcl", "concord_knn", "contrastive"]

# concord_args = {
#     "encoder_dims": [512, 256],
#     "beta": 0.1,
#     "n_epochs": 10
# }

output_dir = '../jobs'
device = 'auto'
conda_env = 'concord_env'
batch_key = 'donor_id'
state_key = 'cell_type'
latent_dim = '50'  # Adjust as needed, but should match the encoder_dims in concord_args

mode = 'local'

subprocess.run([
    "python", "./generate_py_sh_jobs.py",
    "--proj_name", proj_name,
    "--adata_filename", f"{file_name}_preprocessed.h5ad",
    "--methods", *methods,
    "--batch_key", batch_key,
    "--state_key", state_key,
    "--latent_dim", latent_dim,
    "--output_dir", output_dir,
    "--device", device,
    "--conda_env", conda_env,
    "--runtime", "02:00:00",
    "--mode", mode,
    # "--concord_kwargs", json.dumps(concord_args)
], check=True)

In [ ]:
proj_folder = Path(output_dir) / f"benchmark_{proj_name}"
proj_folder.mkdir(parents=True, exist_ok=True)  # defensive

submit_all = proj_folder / f"submit_all_{proj_name}.sh"

with submit_all.open("w") as f:
    f.write("#!/bin/bash\n")
    f.write("# Auto-generated — sequentially runs every job script\n")
    f.write('cd "$(dirname "$0")"\n\n')
    f.write("timestamp=$(date +'%m%d-%H%M')\n\n")

    for sh_file in sorted(proj_folder.glob(f"benchmark_{proj_name}_*.sh")):
        base = sh_file.stem
        f.write(f"echo '🔄 Running: {sh_file.name} (log: {base}_${{timestamp}}.log)'\n")
        f.write(f"bash {sh_file.name} > {base}_${{timestamp}}.log 2>&1\n")
        f.write("echo '✅ Done.'\n\n")

submit_all.chmod(0o755)
print(f"📌  Next step: Run “{submit_all}” to execute all jobs sequentially.")

In [ ]:
# map the obsm back to the adata 
# for each method, uses the most recent run (based on timestamp)
# methods = ["scvi", "harmony", "scanorama", "liger", "unintegrated", "concord_hcl", "concord_knn", "contrastive"]
methods = ["scvi", "harmony", "scanorama", "unintegrated", "concord_hcl", "concord_knn", "contrastive"]

from benchmark_utils import add_embeddings

adata = add_embeddings(adata, proj_name=proj_name, methods=methods)



In [ ]:
adata

In [ ]:
save_dir = Path(f"../save/{proj_name}-{file_suffix}")
save_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
for method in methods:
    if f"{method}_UMAP" in adata.obsm:
        print(f"[⚠️ Warning] obsm['{method}_UMAP'] already exists, skipping UMAP computation")
    else:
        print(f"Computing 2D UMAP for {method}...")
        ccd.ul.run_umap(adata, source_key=f"{method}", result_key=f"{method}_UMAP", n_components=2, n_neighbors=30, min_dist=0.1, metric='euclidean', random_state=seed)
        print(f"✅ obsm['{method}_UMAP'] computed")
    if f"{method}_UMAP_3D" in adata.obsm:
        print(f"[⚠️ Warning] obsm['{method}_UMAP_3D'] already exists, skipping UMAP computation")
    else:
        print(f"Computing 3D UMAP for {method}...")
        ccd.ul.run_umap(adata,source_key=f"{method}",result_key=f"{method}_UMAP_3D",n_components=3,  min_dist=0.1, metric='euclidean', random_state=seed)
        print(f"✅ obsm['{method}_UMAP_3D'] computed")

ccd.ul.save_obsm_to_hdf5(adata, save_dir / f"obsm_{file_suffix}.h5")
print(f"✅ Embeddings saved to {save_dir / f'obsm_{file_suffix}.h5'}")
final_path = data_dir / f"{file_name}_final.h5ad"
adata.write_h5ad(final_path)
print(f"✅ Final AnnData saved to: {final_path}")

In [ ]:
adata

In [ ]:
# plot everything
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import rcParams

# Set DejaVu Sans as the default font
custom_rc = {
    'font.family': 'DejaVu Sans',  # Set the desired font for this plot
}

show_keys = methods
show_cols = ['donor_id', 'cell_type']
basis_types = ['UMAP']

font_size=10
point_size=.1
alpha=0.8
ncols = len(show_keys)
figsize=(ncols * 1.5,1.5)
nrows = int(np.ceil(len(show_keys) / ncols))
pal = {'donor_id': 'Set1', 
       'cell_type': 'Set1'}

with plt.rc_context(rc=custom_rc):
    ccd.pl.plot_all_embeddings(
        adata,
        show_keys,
        color_bys=show_cols,
        basis_types=basis_types,
        pal=pal,
        font_size=font_size,
        point_size=point_size,
        alpha=alpha,
        figsize=figsize,
        ncols=ncols,
        seed=seed,
        save_dir=save_dir,
        file_suffix=file_suffix,
        dpi=600,
        save_format='svg'
    )


In [ ]:
output_key = 'concord_hcl'
basis = output_key
show_basis = basis + '_UMAP'
show_cols = ['donor_id', 'cell_type']
with plt.rc_context(rc=custom_rc):
    ccd.pl.plot_embedding(
        adata, show_basis, show_cols, figsize=(8,4), dpi=600, ncols=2, font_size=4, point_size=0.5, legend_loc='on data',
        pal = pal, seed=seed,
        save_path=save_dir / f"{show_basis}_{file_suffix}_wttext.pdf"
    )

In [ ]:
output_key = 'concord_knn'
basis = output_key
show_basis = basis + '_UMAP'
show_cols = ['donor_id', 'cell_type']
with plt.rc_context(rc=custom_rc):
    ccd.pl.plot_embedding(
        adata, show_basis, show_cols, figsize=(8,4), dpi=600, ncols=2, font_size=4, point_size=0.5, legend_loc='on data',
        pal = pal, seed=seed,
        save_path=save_dir / f"{show_basis}_{file_suffix}_wttext.pdf"
    )

In [ ]:
ccd.ul.anndata_to_viscello(adata, data_dir / f"cello_{proj_name}_{file_suffix}", project_name = proj_name, organism='cel')
print(f"Saved viscello to {data_dir / f'cello_{proj_name}_{file_suffix}'}")

# Benchmarking

In [ ]:
from benchmark_utils import collect_benchmark_logs
bench_df = collect_benchmark_logs("dkd_Wilson", methods)
# Save the benchmark results
bench_df.to_csv(save_dir / f"benchmark_summary_{file_suffix}.tsv", sep="\t", index=False)
print(f"✅ Benchmark summary saved to: {save_dir / f'benchmark_summary_{file_suffix}.tsv'}")
# Plot benchmark results
from benchmark_utils import plot_benchmark_performance
import matplotlib.pyplot as plt
custom_rc = {
    'font.family': 'DejaVu Sans',  # Set the desired font for this plot
}
with plt.rc_context(rc=custom_rc):
    plot_benchmark_performance(bench_df, figsize=(8,2), dpi=300, save_path = save_dir / f"benchmark_plot_{file_suffix}.pdf")


In [ ]:
import scib_metrics
print("scib_metrics version:", scib_metrics.__version__)

In [ ]:
state_key = 'cell_type'
batch_key = 'donor_id'
save_dir = Path(f"../save/{proj_name}-{file_suffix}")
out = ccd.bm.run_benchmark_pipeline(
    adata,
    embedding_keys=methods,
    state_key=state_key,
    batch_key=batch_key,
    save_dir=save_dir / "benchmarks_celltype",
    file_suffix=file_suffix,  # e.g. "2025-06-25"
    run=("probe"),          # run only these blocks
    plot_individual=False,          # skip the intermediate PDFs
)
combined_celltype = out["combined"]

# Save the benchmark results
import pickle
with open(save_dir / f"benchmark_{state_key}_{file_suffix}.pkl", "wb") as f:
    pickle.dump(out, f)

print(f"✅ Benchmark results saved to: {save_dir / f'benchmark_{state_key}_{file_suffix}.pkl'}")

In [ ]:
methods = ["scvi", "harmony", "scanorama", "unintegrated", "concord_hcl", "concord_knn", "contrastive"]
state_key = "cell_type"
batch_key = "donor_id"

In [ ]:
adata = sc.read_h5ad(data_dir / f"{file_name}_final.h5ad")

In [ ]:
from scib_metrics.benchmark import Benchmarker, BioConservation, BatchCorrection

In [ ]:
bm = Benchmarker(
    adata,
    batch_key="donor_id",
    label_key="cell_type",
    bio_conservation_metrics=BioConservation(),
    batch_correction_metrics=BatchCorrection(),
    embedding_obsm_keys=["unintegrated", "scanorama", "harmony", "scvi", "concord_hcl", "concord_knn", "contrastive"],
    n_jobs=4,
)
bm.benchmark()